Dependencies, Imports and util functions

In [ ]:
!pip install gymnasium[box2d]
!pip install swig
!pip install gymnasium[box2d] imageio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 114.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 103.4 MB/s eta 0:00:00


In [ ]:
import gymnasium as gym
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import imageio
import os
from google.colab import files

In [ ]:
# Util functions

def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [ ]:
# Actor Critic Network (shared backbone architecture)
class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, hidden=128):
        super().__init__()

        self.shared = nn.Sequential(
            nn.Linear(state_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
        )

        self.mu_head = nn.Linear(hidden, action_dim)
        self.log_std = nn.Parameter(torch.zeros(action_dim))
        self.value_head = nn.Linear(hidden, 1)

    def forward(self, x):
        z = self.shared(x)
        mu = self.mu_head(z)
        log_std = torch.clamp(self.log_std, -20, 2)
        std = torch.exp(log_std)
        value = self.value_head(z).squeeze(-1)
        return mu, std, value

    def get_action(self, state):
        mu, std, value = self.forward(state)
        dist = torch.distributions.Normal(mu, std)

        u = dist.rsample()
        action = torch.tanh(u)

        logp = dist.log_prob(u)
        logp -= torch.log(1 - action.pow(2) + 1e-6)
        logp = logp.sum(-1)

        entropy = dist.entropy().sum(-1)
        return action, logp, entropy, value

    def evaluate_actions(self, states, actions):
        mu, std, value = self.forward(states)
        dist = torch.distributions.Normal(mu, std)

        eps = 1e-6
        actions = torch.clamp(actions, -1 + eps, 1 - eps)
        u = 0.5 * torch.log((1 + actions) / (1 - actions))

        logp = dist.log_prob(u)
        logp -= torch.log(1 - actions.pow(2) + 1e-6)
        logp = logp.sum(-1)

        entropy = dist.entropy().sum(-1)
        return logp, entropy, value

    def act_deterministic(self, state):
        with torch.no_grad():
            mu, _, _ = self.forward(state)
            return torch.tanh(mu)


# Computing GAE

@torch.no_grad()
def compute_gae(rewards, dones, values, next_value, gamma=0.99, gae_lambda=0.95):
    T = rewards.shape[0]
    advantages = torch.zeros(T, device=rewards.device)

    gae = 0
    for t in reversed(range(T)):
        mask = 1.0 - dones[t]
        v_next = next_value if t == T - 1 else values[t + 1]
        delta = rewards[t] + gamma * v_next * mask - values[t]
        gae = delta + gamma * gae_lambda * mask * gae
        advantages[t] = gae

    returns = advantages + values
    return advantages.detach(), returns.detach()



PPO v1 (without LR annealing and value clipping)

In [ ]:
# Training setup

seed = 0
set_seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

env = gym.make("LunarLanderContinuous-v3")
eval_env = gym.make("LunarLanderContinuous-v3", render_mode="rgb_array")

os.makedirs("videos", exist_ok=True)

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]

model = ActorCritic(state_dim, action_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=3e-4)

# PPO hyperparameters
gamma = 0.99
gae_lambda = 0.95
clip_eps = 0.2
vf_coef = 0.5
ent_coef = 0.005
max_grad_norm = 0.5
ppo_epochs = 10
mini_batch_size = 64

rollout_len = 2048
total_timesteps = 1_000_000
eval_interval = 10_000

global_step = 0
state, _ = env.reset()

# PPO v1 training loop

while global_step < total_timesteps:

    states, actions, rewards, dones, values, logps = [], [], [], [], [], []
    rollout_rewards = []

    # Rollouts
    for _ in range(rollout_len):

        state_tensor = torch.tensor(state, dtype=torch.float32, device=device)
        action, logp, entropy, value = model.get_action(state_tensor)

        next_state, reward, terminated, truncated, _ = env.step(
            action.detach().cpu().numpy()
        )

        done = terminated or truncated

        states.append(state_tensor)
        actions.append(action.detach())
        rewards.append(torch.tensor(reward, dtype=torch.float32, device=device))
        dones.append(torch.tensor(float(done), dtype=torch.float32, device=device))
        values.append(value.detach())
        logps.append(logp.detach())

        rollout_rewards.append(reward)

        state = next_state
        global_step += 1

        if done:
            state, _ = env.reset()

        if global_step >= total_timesteps:
            break

    mean_rollout_reward = np.mean(rollout_rewards)

    states = torch.stack(states)
    actions = torch.stack(actions)
    rewards = torch.stack(rewards)
    dones = torch.stack(dones)
    values = torch.stack(values)
    logps_old = torch.stack(logps)

    with torch.no_grad():
        last_state_tensor = torch.tensor(state, dtype=torch.float32, device=device)
        _, _, next_value = model.forward(last_state_tensor)

    advantages, returns = compute_gae(
        rewards, dones, values, next_value, gamma, gae_lambda
    )

    # Advantage normalisation

    advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)


    dataset_size = states.size(0)

    # Logging metrics
    approx_kl_total = 0
    clip_frac_total = 0
    update_steps = 0

    # PPO update
    for _ in range(ppo_epochs):

        indices = torch.randperm(dataset_size, device=device)

        for start in range(0, dataset_size, mini_batch_size):
            end = start + mini_batch_size
            batch_idx = indices[start:end]

            mb_states = states[batch_idx]
            mb_actions = actions[batch_idx]
            mb_advantages = advantages[batch_idx]
            mb_returns = returns[batch_idx]
            mb_logps_old = logps_old[batch_idx]

            logps_new, entropy, values_new = model.evaluate_actions(
                mb_states, mb_actions
            )

            ratio = torch.exp(logps_new - mb_logps_old)

            surr1 = ratio * mb_advantages
            surr2 = torch.clamp(ratio, 1 - clip_eps, 1 + clip_eps) * mb_advantages  # PPO clipping happens here

            policy_loss = -torch.min(surr1, surr2).mean()
            value_loss = 0.5 * (mb_returns - values_new).pow(2).mean()
            entropy_loss = entropy.mean()

            loss = policy_loss + vf_coef * value_loss - ent_coef * entropy_loss   # Loss formula (Actor loss + Critic loss + entropy term)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)     # Gradient clipping (for stable updates)
            optimizer.step()

            # Logging stats
            approx_kl = (mb_logps_old - logps_new).mean().item()
            clip_frac = ((ratio - 1.0).abs() > clip_eps).float().mean().item()

            approx_kl_total += approx_kl
            clip_frac_total += clip_frac
            update_steps += 1

    # Print logged metrics
    print(
        f"Step: {global_step} | "
        f"Rollout Reward: {mean_rollout_reward:.2f} | "
        f"KL: {approx_kl_total/update_steps:.5f} | "
        f"ClipFrac: {clip_frac_total/update_steps:.3f} | "
        f"Entropy: {entropy_loss.item():.3f}"
    )

    # Evaluation for 1 episode
    if global_step % eval_interval < rollout_len:

        s, _ = eval_env.reset()
        done = False
        ep_return = 0
        frames = []

        while not done:
            s_tensor = torch.tensor(s, dtype=torch.float32, device=device)

            # Deterministic evaluation
            action = model.act_deterministic(s_tensor)

            s, r, terminated, truncated, _ = eval_env.step(
                action.cpu().numpy()
            )
            done = terminated or truncated
            ep_return += r

            frames.append(eval_env.render())

        print(f"[Eval @ {global_step}] Return: {ep_return:.2f}")

        # Saving eval videos
        video_path = f"videos/ppo_eval_{global_step}.mp4"
        imageio.mimsave(video_path, frames, fps=30)
        print(f"Saved video: {video_path}")

env.close()
eval_env.close()


Device: cuda
Step: 2048 | Rollout Reward: -2.02 | KL: 0.01227 | ClipFrac: 0.138 | Entropy: 2.845
Step: 4096 | Rollout Reward: -1.88 | KL: 0.01900 | ClipFrac: 0.191 | Entropy: 2.862
Step: 6144 | Rollout Reward: -2.22 | KL: 0.01231 | ClipFrac: 0.139 | Entropy: 2.861
Step: 8192 | Rollout Reward: -2.98 | KL: 0.01119 | ClipFrac: 0.142 | Entropy: 2.871
Step: 10240 | Rollout Reward: -2.89 | KL: 0.00890 | ClipFrac: 0.156 | Entropy: 2.862
[Eval @ 10240] Return: -323.45


Saved video: videos/ppo_eval_10240.mp4
Step: 12288 | Rollout Reward: -2.45 | KL: 0.01808 | ClipFrac: 0.173 | Entropy: 2.871
Step: 14336 | Rollout Reward: -1.78 | KL: 0.02211 | ClipFrac: 0.165 | Entropy: 2.885
Step: 16384 | Rollout Reward: -1.50 | KL: 0.01488 | ClipFrac: 0.153 | Entropy: 2.896
Step: 18432 | Rollout Reward: -0.96 | KL: 0.01794 | ClipFrac: 0.205 | Entropy: 2.907
Step: 20480 | Rollout Reward: -1.15 | KL: 0.01209 | ClipFrac: 0.112 | Entropy: 2.905


[Eval @ 20480] Return: -30.97
Saved video: videos/ppo_eval_20480.mp4
Step: 22528 | Rollout Reward: -0.77 | KL: 0.01121 | ClipFrac: 0.156 | Entropy: 2.900
Step: 24576 | Rollout Reward: -0.70 | KL: 0.02256 | ClipFrac: 0.216 | Entropy: 2.891
Step: 26624 | Rollout Reward: -0.52 | KL: 0.01297 | ClipFrac: 0.190 | Entropy: 2.875
Step: 28672 | Rollout Reward: -0.47 | KL: 0.01236 | ClipFrac: 0.166 | Entropy: 2.892
Step: 30720 | Rollout Reward: -0.53 | KL: 0.01436 | ClipFrac: 0.179 | Entropy: 2.877


[Eval @ 30720] Return: -42.02
Saved video: videos/ppo_eval_30720.mp4
Step: 32768 | Rollout Reward: -0.47 | KL: 0.01762 | ClipFrac: 0.181 | Entropy: 2.863
Step: 34816 | Rollout Reward: -0.11 | KL: 0.01489 | ClipFrac: 0.155 | Entropy: 2.833
Step: 36864 | Rollout Reward: -0.10 | KL: 0.00973 | ClipFrac: 0.120 | Entropy: 2.804
Step: 38912 | Rollout Reward: -0.12 | KL: 0.01049 | ClipFrac: 0.171 | Entropy: 2.774
Step: 40960 | Rollout Reward: -0.08 | KL: 0.01229 | ClipFrac: 0.155 | Entropy: 2.760


[Eval @ 40960] Return: -149.45
Saved video: videos/ppo_eval_40960.mp4
Step: 43008 | Rollout Reward: -0.18 | KL: 0.01108 | ClipFrac: 0.132 | Entropy: 2.748
Step: 45056 | Rollout Reward: -0.09 | KL: 0.01428 | ClipFrac: 0.119 | Entropy: 2.739
Step: 47104 | Rollout Reward: 0.00 | KL: 0.01018 | ClipFrac: 0.129 | Entropy: 2.749
Step: 49152 | Rollout Reward: -0.05 | KL: 0.00814 | ClipFrac: 0.116 | Entropy: 2.726
Step: 51200 | Rollout Reward: 0.04 | KL: 0.01508 | ClipFrac: 0.176 | Entropy: 2.736


...

[Eval @ 960512] Return: 286.10
Saved video: videos/ppo_eval_960512.mp4
Step: 962560 | Rollout Reward: 0.59 | KL: 0.02146 | ClipFrac: 0.215 | Entropy: 0.828
Step: 964608 | Rollout Reward: 0.58 | KL: 0.02892 | ClipFrac: 0.284 | Entropy: 0.825
Step: 966656 | Rollout Reward: 0.57 | KL: 0.02493 | ClipFrac: 0.243 | Entropy: 0.820
Step: 968704 | Rollout Reward: 0.44 | KL: 0.04417 | ClipFrac: 0.365 | Entropy: 0.827
Step: 970752 | Rollout Reward: 0.66 | KL: 0.02394 | ClipFrac: 0.226 | Entropy: 0.809


[Eval @ 970752] Return: 278.91
Saved video: videos/ppo_eval_970752.mp4
Step: 972800 | Rollout Reward: 0.78 | KL: 0.02072 | ClipFrac: 0.194 | Entropy: 0.804
Step: 974848 | Rollout Reward: 0.73 | KL: 0.03107 | ClipFrac: 0.245 | Entropy: 0.803
Step: 976896 | Rollout Reward: 0.52 | KL: 0.01933 | ClipFrac: 0.226 | Entropy: 0.790
Step: 978944 | Rollout Reward: 0.82 | KL: 0.01723 | ClipFrac: 0.218 | Entropy: 0.767
Step: 980992 | Rollout Reward: 1.02 | KL: 0.02655 | ClipFrac: 0.259 | Entropy: 0.764


[Eval @ 980992] Return: 281.88
Saved video: videos/ppo_eval_980992.mp4
Step: 983040 | Rollout Reward: 0.75 | KL: 0.01377 | ClipFrac: 0.203 | Entropy: 0.779
Step: 985088 | Rollout Reward: 0.26 | KL: 0.03750 | ClipFrac: 0.315 | Entropy: 0.787
Step: 987136 | Rollout Reward: 0.52 | KL: 0.03474 | ClipFrac: 0.273 | Entropy: 0.768
Step: 989184 | Rollout Reward: 0.83 | KL: 0.02513 | ClipFrac: 0.236 | Entropy: 0.775
Step: 991232 | Rollout Reward: 0.83 | KL: 0.02950 | ClipFrac: 0.245 | Entropy: 0.790


[Eval @ 991232] Return: 253.58
Saved video: videos/ppo_eval_991232.mp4
Step: 993280 | Rollout Reward: 0.51 | KL: 0.06855 | ClipFrac: 0.338 | Entropy: 0.789
Step: 995328 | Rollout Reward: 0.46 | KL: 0.05297 | ClipFrac: 0.286 | Entropy: 0.787
Step: 997376 | Rollout Reward: 1.05 | KL: 0.07104 | ClipFrac: 0.351 | Entropy: 0.797
Step: 999424 | Rollout Reward: 0.64 | KL: 0.03417 | ClipFrac: 0.319 | Entropy: 0.769
Step: 1000000 | Rollout Reward: -0.01 | KL: 0.02816 | ClipFrac: 0.351 | Entropy: 0.777


[Eval @ 1000000] Return: 263.20
Saved video: videos/ppo_eval_1000000.mp4


In [ ]:
!zip -r videos.zip /content/videos

  adding: content/videos/ (stored 0%)
  adding: content/videos/ppo_eval_61440.mp4 (deflated 13%)
  adding: content/videos/ppo_eval_100352.mp4 (deflated 17%)
  adding: content/videos/ppo_eval_20480.mp4 (deflated 11%)
  adding: content/videos/ppo_eval_620544.mp4 (deflated 14%)
  adding: content/videos/ppo_eval_141312.mp4 (deflated 11%)
  adding: content/videos/ppo_eval_901120.mp4 (deflated 13%)
  adding: content/videos/ppo_eval_512000.mp4 (deflated 9%)
  adding: content/videos/ppo_eval_581632.mp4 (deflated 23%)
  adding: content/videos/ppo_eval_311296.mp4 (deflated 14%)
  adding: content/videos/ppo_eval_270336.mp4 (deflated 14%)
  adding: content/videos/ppo_eval_870400.mp4 (deflated 15%)
  adding: content/videos/ppo_eval_591872.mp4 (deflated 17%)
  adding: content/videos/ppo_eval_81920.mp4 (deflated 13%)
  adding: content/videos/ppo_eval_260096.mp4 (deflated 16%)
  adding: content/videos/ppo_eval_561152.mp4 (deflated 14%)
  adding: content/videos/ppo_eval_790528.mp4 (deflated 14%)
  addi

In [ ]:
files.download('/content/videos.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PPO v2 (with LR annealing ,value clipping and additional logging metrics)

In [ ]:
# Training setup

seed = 0
set_seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

env = gym.make("LunarLanderContinuous-v3")
eval_env = gym.make("LunarLanderContinuous-v3", render_mode="rgb_array")

os.makedirs("videos_v2", exist_ok=True)

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]

initial_lr = 3e-4   # LR annealing initial LR
model = ActorCritic(state_dim, action_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=initial_lr)

# PPO hyperparameters
gamma = 0.99
gae_lambda = 0.95
clip_eps = 0.2
vf_coef = 0.5
ent_coef = 0.005
max_grad_norm = 0.5
ppo_epochs = 10
mini_batch_size = 64

rollout_len = 2048
total_timesteps = 1_000_000
eval_interval = 10_000

global_step = 0
state, _ = env.reset()

# PPO v2 training loop

while global_step < total_timesteps:

    # LR Annealing
    frac = 1.0 - (global_step / total_timesteps)
    lr_now = initial_lr * frac
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr_now

    states, actions, rewards, dones, values, logps = [], [], [], [], [], []
    rollout_rewards = []

    # Rollouts
    for _ in range(rollout_len):

        state_tensor = torch.tensor(state, dtype=torch.float32, device=device)
        action, logp, entropy, value = model.get_action(state_tensor)

        next_state, reward, terminated, truncated, _ = env.step(
            action.detach().cpu().numpy()
        )

        done = terminated or truncated

        states.append(state_tensor)
        actions.append(action.detach())
        rewards.append(torch.tensor(reward, dtype=torch.float32, device=device))
        dones.append(torch.tensor(float(done), dtype=torch.float32, device=device))
        values.append(value.detach())
        logps.append(logp.detach())

        rollout_rewards.append(reward)

        state = next_state
        global_step += 1

        if done:
            state, _ = env.reset()

        if global_step >= total_timesteps:
            break

    mean_rollout_reward = np.mean(rollout_rewards)

    states = torch.stack(states)
    actions = torch.stack(actions)
    rewards = torch.stack(rewards)
    dones = torch.stack(dones)
    values = torch.stack(values)
    logps_old = torch.stack(logps)

    with torch.no_grad():
        last_state_tensor = torch.tensor(state, dtype=torch.float32, device=device)
        _, _, next_value = model.forward(last_state_tensor)

    advantages, returns = compute_gae(
        rewards, dones, values, next_value, gamma, gae_lambda
    )

    # Advantage Norm
    advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)


    dataset_size = states.size(0)

    # Logging metrics
    approx_kl_total = 0
    clip_frac_total = 0
    explained_var_total = 0
    update_steps = 0

    # PPO V2 main training loop
    for _ in range(ppo_epochs):

        indices = torch.randperm(dataset_size, device=device)

        for start in range(0, dataset_size, mini_batch_size):
            end = start + mini_batch_size
            batch_idx = indices[start:end]

            mb_states = states[batch_idx]
            mb_actions = actions[batch_idx]
            mb_advantages = advantages[batch_idx]
            mb_returns = returns[batch_idx]
            mb_logps_old = logps_old[batch_idx]
            mb_values_old = values[batch_idx]

            logps_new, entropy, values_new = model.evaluate_actions(
                mb_states, mb_actions
            )

            ratio = torch.exp(logps_new - mb_logps_old)

            surr1 = ratio * mb_advantages
            surr2 = torch.clamp(ratio, 1 - clip_eps, 1 + clip_eps) * mb_advantages   # PPO clipping

            policy_loss = -torch.min(surr1, surr2).mean()

            # Value function clipping
            value_clipped = mb_values_old + torch.clamp(
                values_new - mb_values_old,
                -clip_eps,
                clip_eps,
            )

            v_loss1 = (values_new - mb_returns).pow(2)
            v_loss2 = (value_clipped - mb_returns).pow(2)
            value_loss = 0.5 * torch.max(v_loss1, v_loss2).mean()

            entropy_loss = entropy.mean()

            # PPO Loss
            loss = policy_loss + vf_coef * value_loss - ent_coef * entropy_loss

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Gradient clipping
            optimizer.step()

            # Logging stats
            approx_kl = (mb_logps_old - logps_new).mean().item()
            clip_frac = ((ratio - 1.0).abs() > clip_eps).float().mean().item()

            explained_var = 1 - torch.var(mb_returns - values_new) / (
                torch.var(mb_returns) + 1e-8
            )
            explained_var_total += explained_var.item()

            approx_kl_total += approx_kl
            clip_frac_total += clip_frac
            update_steps += 1

    print(
        f"Step: {global_step} | "
        f"LR: {lr_now:.6f} | "
        f"Reward: {mean_rollout_reward:.2f} | "
        f"KL: {approx_kl_total/update_steps:.5f} | "
        f"ClipFrac: {clip_frac_total/update_steps:.3f} | "
        f"ExplVar: {explained_var_total/update_steps:.3f}"
    )

    # Evaluation
    if global_step % eval_interval < rollout_len:

        s, _ = eval_env.reset()
        done = False
        ep_return = 0
        frames = []

        while not done:
            s_tensor = torch.tensor(s, dtype=torch.float32, device=device)

            # Determininstic evaluation
            action = model.act_deterministic(s_tensor)

            s, r, terminated, truncated, _ = eval_env.step(
                action.cpu().numpy()
            )
            done = terminated or truncated
            ep_return += r

            frames.append(eval_env.render())

        print(f"[Eval @ {global_step}] Return: {ep_return:.2f}")

        video_path = f"videos_v2/ppo_eval_{global_step}.mp4"
        imageio.mimsave(video_path, frames, fps=30)
        print(f"Saved video: {video_path}")

env.close()
eval_env.close()


Device: cuda
Step: 2048 | LR: 0.000300 | Reward: -1.38 | KL: 0.00497 | ClipFrac: 0.034 | ExplVar: 0.006
Step: 4096 | LR: 0.000299 | Reward: -1.28 | KL: 0.00492 | ClipFrac: 0.046 | ExplVar: 0.018
Step: 6144 | LR: 0.000299 | Reward: -0.48 | KL: 0.00471 | ClipFrac: 0.066 | ExplVar: 0.032
Step: 8192 | LR: 0.000298 | Reward: -1.00 | KL: 0.00739 | ClipFrac: 0.039 | ExplVar: 0.035
Step: 10240 | LR: 0.000298 | Reward: -0.77 | KL: 0.00129 | ClipFrac: 0.026 | ExplVar: 0.043


[Eval @ 10240] Return: -116.26
Saved video: videos_v2/ppo_eval_10240.mp4
Step: 12288 | LR: 0.000297 | Reward: -0.69 | KL: 0.01074 | ClipFrac: 0.059 | ExplVar: 0.064
Step: 14336 | LR: 0.000296 | Reward: -0.68 | KL: 0.00373 | ClipFrac: 0.050 | ExplVar: 0.081
Step: 16384 | LR: 0.000296 | Reward: -0.62 | KL: 0.00788 | ClipFrac: 0.066 | ExplVar: 0.107
Step: 18432 | LR: 0.000295 | Reward: -0.43 | KL: 0.01254 | ClipFrac: 0.078 | ExplVar: 0.148
Step: 20480 | LR: 0.000294 | Reward: -0.57 | KL: 0.01079 | ClipFrac: 0.056 | ExplVar: 0.128


[Eval @ 20480] Return: -130.31
Saved video: videos_v2/ppo_eval_20480.mp4
Step: 22528 | LR: 0.000294 | Reward: -0.14 | KL: 0.00166 | ClipFrac: 0.041 | ExplVar: 0.167
Step: 24576 | LR: 0.000293 | Reward: -0.12 | KL: 0.00186 | ClipFrac: 0.080 | ExplVar: 0.187
Step: 26624 | LR: 0.000293 | Reward: -0.29 | KL: 0.00689 | ClipFrac: 0.047 | ExplVar: 0.204
Step: 28672 | LR: 0.000292 | Reward: -0.01 | KL: 0.00623 | ClipFrac: 0.063 | ExplVar: 0.214
Step: 30720 | LR: 0.000291 | Reward: -0.03 | KL: 0.00727 | ClipFrac: 0.065 | ExplVar: 0.337


[Eval @ 30720] Return: -161.09
Saved video: videos_v2/ppo_eval_30720.mp4
Step: 32768 | LR: 0.000291 | Reward: 0.01 | KL: 0.00261 | ClipFrac: 0.060 | ExplVar: 0.370
Step: 34816 | LR: 0.000290 | Reward: -0.04 | KL: 0.00516 | ClipFrac: 0.090 | ExplVar: 0.367
Step: 36864 | LR: 0.000290 | Reward: 0.00 | KL: 0.00884 | ClipFrac: 0.085 | ExplVar: 0.461
Step: 38912 | LR: 0.000289 | Reward: -0.03 | KL: 0.00944 | ClipFrac: 0.094 | ExplVar: 0.366
Step: 40960 | LR: 0.000288 | Reward: 0.05 | KL: 0.00195 | ClipFrac: 0.066 | ExplVar: 0.618


[Eval @ 40960] Return: 114.11
Saved video: videos_v2/ppo_eval_40960.mp4
Step: 43008 | LR: 0.000288 | Reward: 0.08 | KL: 0.00531 | ClipFrac: 0.086 | ExplVar: 0.664
Step: 45056 | LR: 0.000287 | Reward: 0.10 | KL: 0.00363 | ClipFrac: 0.094 | ExplVar: 0.680
Step: 47104 | LR: 0.000286 | Reward: 0.10 | KL: 0.00393 | ClipFrac: 0.064 | ExplVar: 0.631
Step: 49152 | LR: 0.000286 | Reward: 0.12 | KL: 0.00803 | ClipFrac: 0.064 | ExplVar: 0.520
Step: 51200 | LR: 0.000285 | Reward: 0.10 | KL: 0.00449 | ClipFrac: 0.061 | ExplVar: 0.551


...

[Eval @ 960512] Return: 262.03
Saved video: videos_v2/ppo_eval_960512.mp4
Step: 962560 | LR: 0.000012 | Reward: 1.21 | KL: 0.00217 | ClipFrac: 0.014 | ExplVar: 0.880
Step: 964608 | LR: 0.000011 | Reward: 0.77 | KL: 0.00191 | ClipFrac: 0.038 | ExplVar: 0.915
Step: 966656 | LR: 0.000011 | Reward: 0.68 | KL: 0.00086 | ClipFrac: 0.022 | ExplVar: 0.788
Step: 968704 | LR: 0.000010 | Reward: 1.23 | KL: 0.00292 | ClipFrac: 0.006 | ExplVar: 0.824
Step: 970752 | LR: 0.000009 | Reward: 0.63 | KL: -0.00099 | ClipFrac: 0.010 | ExplVar: 0.861


[Eval @ 970752] Return: 294.51
Saved video: videos_v2/ppo_eval_970752.mp4
Step: 972800 | LR: 0.000009 | Reward: 0.42 | KL: 0.00029 | ClipFrac: 0.001 | ExplVar: 0.940
Step: 974848 | LR: 0.000008 | Reward: 0.24 | KL: 0.00262 | ClipFrac: 0.021 | ExplVar: 0.846
Step: 976896 | LR: 0.000008 | Reward: 0.61 | KL: -0.00000 | ClipFrac: 0.000 | ExplVar: 0.777
Step: 978944 | LR: 0.000007 | Reward: 0.78 | KL: 0.00054 | ClipFrac: 0.000 | ExplVar: 0.776
Step: 980992 | LR: 0.000006 | Reward: 0.64 | KL: 0.00026 | ClipFrac: 0.000 | ExplVar: 0.798


[Eval @ 980992] Return: 287.29
Saved video: videos_v2/ppo_eval_980992.mp4
Step: 983040 | LR: 0.000006 | Reward: 0.84 | KL: 0.00115 | ClipFrac: 0.001 | ExplVar: 0.799
Step: 985088 | LR: 0.000005 | Reward: 1.00 | KL: 0.00011 | ClipFrac: 0.000 | ExplVar: 0.721
Step: 987136 | LR: 0.000004 | Reward: 0.66 | KL: 0.00288 | ClipFrac: 0.019 | ExplVar: 0.812
Step: 989184 | LR: 0.000004 | Reward: 0.90 | KL: 0.00321 | ClipFrac: 0.010 | ExplVar: 0.895
Step: 991232 | LR: 0.000003 | Reward: 0.51 | KL: -0.00018 | ClipFrac: 0.004 | ExplVar: 0.858


[Eval @ 991232] Return: 228.75
Saved video: videos_v2/ppo_eval_991232.mp4
Step: 993280 | LR: 0.000003 | Reward: 0.58 | KL: -0.00036 | ClipFrac: 0.000 | ExplVar: 0.881
Step: 995328 | LR: 0.000002 | Reward: 0.93 | KL: 0.00042 | ClipFrac: 0.000 | ExplVar: 0.881
Step: 997376 | LR: 0.000001 | Reward: 0.57 | KL: 0.00062 | ClipFrac: 0.000 | ExplVar: 0.885
Step: 999424 | LR: 0.000001 | Reward: 0.30 | KL: 0.00029 | ClipFrac: 0.000 | ExplVar: 0.932
Step: 1000000 | LR: 0.000000 | Reward: 1.34 | KL: -0.00006 | ClipFrac: 0.000 | ExplVar: 0.879


[Eval @ 1000000] Return: 255.50
Saved video: videos_v2/ppo_eval_1000000.mp4


In [ ]:
!zip -r videos_v2.zip /content/videos_v2

In [ ]:
files.download('/content/videos_v2.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>